In [1]:
import numpy as np
import scipy.io as sio
import os

# 导入独热编码函数
from tensorflow.keras.utils import to_categorical

# 导入训练测试集划分函数
from sklearn.model_selection import train_test_split

# 导入深度学习相关库
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM
from keras.models import Sequential, Model
import keras
from keras import backend as K
import time

# 导入分层K折交叉验证，保证结果的准确率更加稳定
from sklearn.model_selection import StratifiedKFold

# 设置超参数
num_classes = 2  # 每个情绪类别是一个二分类
batch_size = 128
img_rows, img_cols, num_chan = 8, 9, 4

# a相当于arousal唤醒，v相当于valence效价
flag = 'a'
# flag = 'v'   # 这里需要注意，当你第一次训练之后，需要把flag = 'a'注释掉，用flag = 'v'重新训练一遍
t = 6

# 存放结果
acc_list = []
std_list = []
all_acc = []

# 被试者ID
short_name = ['01', '02', '03', '04', '05', '06', '07', '08', '09',
              '10', '11', '12', '13', '14', '15', '16', '17', '18',
              '19', '20', '21', '22', '23', '24', '25', '26', '27',
              '28', '29', '30', '31', '32']

# 对每个被试分别进行交叉验证，取平均
dataset_dir = "../middle/with_base_0.5/"
for i in range(len(short_name)):
    # 释放内存 
    K.clear_session()
    start = time.time()
    # print("\nprocessing: ", short_name[i], "......")
    
    # 读入数据
    file_path = os.path.join(dataset_dir, 'DE_s'+short_name[i])
    file = sio.loadmat(file_path)
    data = file['data']  # shape为4800 *4 * 8 * 9
    
    # 独热编码
    y_v = file['valence_labels'][0]
    y_a = file['arousal_labels'][0]
    
    
    y_v = to_categorical(y_v, num_classes)   # num_classes是类别个数，独热编码的结果是几个类别就是几
    y_a = to_categorical(y_a, num_classes)
    
    # 调整数据格式，即调整数据维度
    one_falx = data.transpose([0, 2, 3, 1])
    # one_falx = one_falx[:,:,:,2:4]
    # 继续调整数据维度
    one_falx = one_falx.reshape((-1, t, img_rows, img_cols, num_chan))  
    
    # 构造标签
    one_y_v = np.empty([0,2])
    one_y_a = np.empty([0,2])
    for j in range(int(len(y_a)//t)):
        one_y_v = np.vstack((one_y_v, y_v[j*t]))
        one_y_a = np.vstack((one_y_a, y_a[j*t]))
    # print(one_y_v.shape)
    # print(one_y_a.shape)
    # print(one_falx.shape)

    # 选择情感维度
    if flag=='v':
        one_y = one_y_v
    else:
        one_y = one_y_a

    # 5折交叉验证
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)   # 交叉验证为5
    # StratifiedKFold这个函数好像是分层抽样，比如有1,2,3,4 4个数的时候，第一次抽1，2，下次就会抽3，4
    # 所以这里分为6则交叉验证，label也要按照6的分层来设计，也就是每隔6个，这6个相同，然后下一组6个也要相同
    # x_train, x_test, y_train, y_test = train_test_split(one_falx, one_y, test_size=0.1)
    cvscores = []

    # 构建模型
    for train, test in kfold.split(one_falx, one_y.argmax(1)):
        img_size = (img_rows, img_cols, num_chan)
        # print("train.shape:",train.shape) 这里是640
        # print("train.shape:", test.shape) 这里是160,我明白了是在每一个800的组里面进行5则交叉验证
        
        # 构建深度学习网络，CNN层+LSTM层
        def create_base_network(input_dim):
            seq = Sequential()
            seq.add(Conv2D(64, 5, activation='relu', padding='same', name='conv1', input_shape=input_dim))
            seq.add(Conv2D(128, 4, activation='relu', padding='same', name='conv2'))
            seq.add(Conv2D(256, 4, activation='relu', padding='same', name='conv3'))
            seq.add(Conv2D(64, 1, activation='relu', padding='same', name='conv4'))
            seq.add(MaxPooling2D(2, 2, name='pool1'))
            seq.add(Flatten(name='fla1'))
            seq.add(Dense(512, activation='relu', name='dense1'))
            seq.add(Reshape((1, 512), name='reshape'))
            return seq


        # 调用函数构建基础网络，即初始化
        base_network = create_base_network(img_size)
        # 构建多输入模型，即LSTM网络
        input_1 = Input(shape=img_size)
        input_2 = Input(shape=img_size)
        input_3 = Input(shape=img_size)
        input_4 = Input(shape=img_size)
        input_5 = Input(shape=img_size)
        input_6 = Input(shape=img_size)

        out_all = Concatenate(axis=1)([base_network(input_1), base_network(input_2), base_network(input_3), base_network(input_4), base_network(input_5), base_network(input_6)])
        lstm_layer = LSTM(128, name='lstm')(out_all)
        out_layer = Dense(2, activation='softmax', name='out')(lstm_layer)
        model = Model([input_1, input_2, input_3, input_4, input_5, input_6], out_layer)
        # model.summary()

        # 编译
        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.adam_v2.Adam(),
                      metrics=['accuracy'])
        # Fit the model
        # one_falx本身的shape是(800,6,8,9,4),而这里x_train的shape变成了(640,6,8,9,4)
        # 这个意思就是索引只在第一个维度进行索引，其他维度不变
        x_train = one_falx[train]
        # print(x_train.shape) 640,6,8,9,4
        # print(x_train[:, 0].shape,x_train[:, 1].shape,x_train[:, 2].shape,x_train[:, 3].shape,x_train[:, 4].shape,x_train[:, 5].shape)
        # 都是640,8,9,4

        y_train = one_y[train]
        #print(y_train.shape) 640,2

        model.fit([x_train[:, 0], x_train[:, 1], x_train[:, 2], x_train[:, 3], x_train[:, 4], x_train[:, 5]], y_train, epochs=100, batch_size=64, verbose=1)
        # 这里的x_train[:,0/1/2/3/4/5]对应的ytrain是同一个值，因为每隔120个点才会变化一次标签，而6是120的公因子，所以这6个训练集的标签是一样的
        # 下面的evaluate同理
        # 训练过程中，5则交叉验证，每一则100个epochs，所以会输出五个准确率的值


        # 评估
        x_test = one_falx[test]
        y_test = one_y[test]
        scores = model.evaluate([x_test[:, 0], x_test[:, 1], x_test[:, 2], x_test[:, 3], x_test[:, 4], x_test[:, 5]], y_test, verbose=0)

        # 存储这次交叉验证的结果
        print("%.2f%%" % (scores[1] * 100))
        all_acc.append(scores[1] * 100)

    # 汇总当前被试的结果
    # print("all acc: {}".format(all_acc))
    print("mean acc: {}".format(np.mean(all_acc)))
    print("std acc: {}".format(np.std(all_acc)))
    acc_list.append(np.mean(all_acc))
    std_list.append(np.std(all_acc))
    print("进度： {}".format(short_name[i]))
    all_acc = []
    end = time.time()
    print("%.2f" % (end - start))
    
# 输出最终所有被试的结果
print('Acc_all: {}'.format(acc_list))
print('Std_all: {}'.format(std_list))
print('Acc_avg: {}'.format(np.mean(acc_list)))
print('Std_avg: {}'.format(np.mean(std_list)))

2023-11-06 19:33:35.377840: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-06 19:33:35.384357: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/conda/miniconda3/envs/czq2/lib/python3.10/site-packages/torch/lib/:/home/user/conda/miniconda3/envs/czq2/lib/python3.10/site-packages/torch/lib/::/usr/local/cuda-11.0/lib64
2023-11-06 19:33:35.385056: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your 

Epoch 1/100
10/10 [==============================] - 5s 270ms/step - loss: 0.7429 - accuracy: 0.5500
Epoch 2/100
10/10 [==============================] - 3s 261ms/step - loss: 0.6738 - accuracy: 0.6000
Epoch 3/100
10/10 [==============================] - 3s 273ms/step - loss: 0.6514 - accuracy: 0.6000
Epoch 4/100
10/10 [==============================] - 3s 270ms/step - loss: 0.5841 - accuracy: 0.6844
Epoch 5/100
10/10 [==============================] - 3s 269ms/step - loss: 0.4914 - accuracy: 0.7594
Epoch 6/100
10/10 [==============================] - 3s 269ms/step - loss: 0.3447 - accuracy: 0.8625
Epoch 7/100
10/10 [==============================] - 3s 264ms/step - loss: 0.2631 - accuracy: 0.8938
Epoch 8/100
10/10 [==============================] - 3s 261ms/step - loss: 0.2188 - accuracy: 0.9250
Epoch 9/100
10/10 [==============================] - 3s 268ms/step - loss: 0.0685 - accuracy: 0.9781
Epoch 10/100
10/10 [==============================] - 3s 258ms/step - loss: 0.0609 - accura

10/10 [==============================] - 3s 274ms/step - loss: 2.5909e-07 - accuracy: 1.0000
Epoch 80/100
10/10 [==============================] - 3s 263ms/step - loss: 2.5220e-07 - accuracy: 1.0000
Epoch 81/100
10/10 [==============================] - 3s 269ms/step - loss: 2.4512e-07 - accuracy: 1.0000
Epoch 82/100
10/10 [==============================] - 3s 266ms/step - loss: 2.4065e-07 - accuracy: 1.0000
Epoch 83/100
10/10 [==============================] - 3s 265ms/step - loss: 2.3358e-07 - accuracy: 1.0000
Epoch 84/100
10/10 [==============================] - 3s 262ms/step - loss: 2.2929e-07 - accuracy: 1.0000
Epoch 85/100
10/10 [==============================] - 3s 264ms/step - loss: 2.2557e-07 - accuracy: 1.0000
Epoch 86/100
10/10 [==============================] - 3s 267ms/step - loss: 2.2110e-07 - accuracy: 1.0000
Epoch 87/100
10/10 [==============================] - 3s 263ms/step - loss: 2.1700e-07 - accuracy: 1.0000
Epoch 88/100
10/10 [==============================] - 3s 27

10/10 [==============================] - 3s 259ms/step - loss: 1.6961e-06 - accuracy: 1.0000
Epoch 58/100
10/10 [==============================] - 3s 265ms/step - loss: 1.6415e-06 - accuracy: 1.0000
Epoch 59/100
10/10 [==============================] - 3s 271ms/step - loss: 1.5907e-06 - accuracy: 1.0000
Epoch 60/100
10/10 [==============================] - 3s 268ms/step - loss: 1.5428e-06 - accuracy: 1.0000
Epoch 61/100
10/10 [==============================] - 3s 264ms/step - loss: 1.4968e-06 - accuracy: 1.0000
Epoch 62/100
10/10 [==============================] - 3s 269ms/step - loss: 1.4521e-06 - accuracy: 1.0000
Epoch 63/100
10/10 [==============================] - 3s 272ms/step - loss: 1.4074e-06 - accuracy: 1.0000
Epoch 64/100
10/10 [==============================] - 3s 265ms/step - loss: 1.3670e-06 - accuracy: 1.0000
Epoch 65/100
10/10 [==============================] - 3s 264ms/step - loss: 1.3309e-06 - accuracy: 1.0000
Epoch 66/100
10/10 [==============================] - 3s 26

10/10 [==============================] - 3s 264ms/step - loss: 8.6877e-06 - accuracy: 1.0000
Epoch 35/100
10/10 [==============================] - 3s 271ms/step - loss: 8.2368e-06 - accuracy: 1.0000
Epoch 36/100
10/10 [==============================] - 3s 273ms/step - loss: 7.8183e-06 - accuracy: 1.0000
Epoch 37/100
10/10 [==============================] - 3s 269ms/step - loss: 7.4294e-06 - accuracy: 1.0000
Epoch 38/100
10/10 [==============================] - 3s 260ms/step - loss: 7.0660e-06 - accuracy: 1.0000
Epoch 39/100
10/10 [==============================] - 3s 260ms/step - loss: 6.7378e-06 - accuracy: 1.0000
Epoch 40/100
10/10 [==============================] - 3s 267ms/step - loss: 6.4072e-06 - accuracy: 1.0000
Epoch 41/100
10/10 [==============================] - 3s 267ms/step - loss: 6.1028e-06 - accuracy: 1.0000
Epoch 42/100
10/10 [==============================] - 3s 262ms/step - loss: 5.8246e-06 - accuracy: 1.0000
Epoch 43/100
10/10 [==============================] - 3s 26

10/10 [==============================] - 3s 255ms/step - loss: 0.0171 - accuracy: 0.9922
Epoch 12/100
10/10 [==============================] - 3s 260ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 13/100
10/10 [==============================] - 3s 264ms/step - loss: 0.0100 - accuracy: 0.9953
Epoch 14/100
10/10 [==============================] - 3s 271ms/step - loss: 0.0075 - accuracy: 0.9969
Epoch 15/100
10/10 [==============================] - 3s 263ms/step - loss: 0.0262 - accuracy: 0.9906
Epoch 16/100
10/10 [==============================] - 3s 265ms/step - loss: 0.0331 - accuracy: 0.9891
Epoch 17/100
10/10 [==============================] - 3s 259ms/step - loss: 0.0315 - accuracy: 0.9937
Epoch 18/100
10/10 [==============================] - 3s 262ms/step - loss: 0.0041 - accuracy: 1.0000
Epoch 19/100
10/10 [==============================] - 3s 256ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 20/100
10/10 [==============================] - 3s 262ms/step - loss: 0.0010 - accuracy: 

10/10 [==============================] - 3s 263ms/step - loss: 3.4291e-07 - accuracy: 1.0000
Epoch 89/100
10/10 [==============================] - 3s 262ms/step - loss: 3.3434e-07 - accuracy: 1.0000
Epoch 90/100
10/10 [==============================] - 3s 260ms/step - loss: 3.2540e-07 - accuracy: 1.0000
Epoch 91/100
10/10 [==============================] - 3s 255ms/step - loss: 3.1870e-07 - accuracy: 1.0000
Epoch 92/100
10/10 [==============================] - 3s 267ms/step - loss: 3.1013e-07 - accuracy: 1.0000
Epoch 93/100
10/10 [==============================] - 3s 265ms/step - loss: 3.0342e-07 - accuracy: 1.0000
Epoch 94/100
10/10 [==============================] - 3s 262ms/step - loss: 2.9635e-07 - accuracy: 1.0000
Epoch 95/100
10/10 [==============================] - 3s 265ms/step - loss: 2.8964e-07 - accuracy: 1.0000
Epoch 96/100
10/10 [==============================] - 3s 269ms/step - loss: 2.8294e-07 - accuracy: 1.0000
Epoch 97/100
10/10 [==============================] - 3s 26

10/10 [==============================] - 3s 267ms/step - loss: 1.0367e-06 - accuracy: 1.0000
Epoch 66/100
10/10 [==============================] - 3s 266ms/step - loss: 1.0023e-06 - accuracy: 1.0000
Epoch 67/100
10/10 [==============================] - 3s 259ms/step - loss: 9.6950e-07 - accuracy: 1.0000
Epoch 68/100
10/10 [==============================] - 3s 255ms/step - loss: 9.3765e-07 - accuracy: 1.0000
Epoch 69/100
10/10 [==============================] - 3s 269ms/step - loss: 9.1437e-07 - accuracy: 1.0000
Epoch 70/100
10/10 [==============================] - 3s 263ms/step - loss: 8.8456e-07 - accuracy: 1.0000
Epoch 71/100
10/10 [==============================] - 3s 262ms/step - loss: 8.5476e-07 - accuracy: 1.0000
Epoch 72/100
10/10 [==============================] - 3s 272ms/step - loss: 8.3111e-07 - accuracy: 1.0000
Epoch 73/100
10/10 [==============================] - 3s 262ms/step - loss: 8.0429e-07 - accuracy: 1.0000
Epoch 74/100
10/10 [==============================] - 3s 26

KeyboardInterrupt: 